# Neural networks code and tests notebook

# This notebook takes the modified raw data and try to predict classes on  :

# - Conv1D network

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame


In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)

    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:
Feature_time_series=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

In [6]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Class_ON = []

nbfeatures=2

a=binning
b=nbfeatures
c=lg

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j])
    dataframe.columns=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['fratio']
    bigdata[2][j]=dataframe['Delta Flux']*dataframe['Flux']
    bigdata[3][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[4][j]=dataframe['fratio']*dataframe['Photon Index']
    bigdata[5][j]=dataframe['Photon Index']
    bigdata[6][j]=abs(dataframe['Photon Index'])-dataframe['fratio']
    bigdata[7][j]=np.var(dataframe['Photon Index'])*dataframe['Photon Index'] 
    bigdata[8][j]=np.var(dataframe['fratio'])*dataframe['fratio'] 

for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j])
    dataframe.columns=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][v]=dataframe['Flux']
    bigdata[1][v]=dataframe['fratio']  
    bigdata[2][v]=dataframe['Delta Flux']*dataframe['Flux']
    bigdata[3][v]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[4][v]=dataframe['fratio']*dataframe['Photon Index']
    bigdata[5][v]=dataframe['Photon Index']
    bigdata[6][v]=abs(dataframe['Photon Index'])-dataframe['fratio']
    bigdata[7][v]=np.var(dataframe['Photon Index'])*dataframe['Photon Index']
    bigdata[8][v]=np.var(dataframe['fratio'])*dataframe['fratio'] 
    


#Creating labels
for i in range(lgOFF):
    
    Class_ON.append(int(0))
    
for i in range(lgON):
    Class_ON.append(int(1))


In [7]:
#Reshaping data matrix
bigdata=bigdata.reshape(c,nbfeatures,binning)

In [14]:
# Conv1D architecture
from keras.layers import GlobalAveragePooling1D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Conv1D

csv_logger = CSVLogger(model_history4, append=True)
weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
bigdata=bigdata.reshape(c,nbfeatures,binning)
nb_filters = 20
fsize=1
x_train, x_test, y_train, y_test = train_test_split(bigdata, Class_ON, test_size=0.3, random_state=38)
y_test2=y_test.copy()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



model = Sequential()
model.add(Conv1D(nb_filters, 1, padding="same", activation="relu",input_shape=(nbfeatures,binning)))
model.add(Conv1D(15, 1, padding="same", activation="relu",input_shape=(nb_filters,binning)))
model.add(Conv1D(9, 1, padding="same", activation="relu",input_shape=(15,binning)))
model.add(GlobalMaxPooling1D())
model.add(Dense(binning, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train,epochs=160,batch_size=4,class_weight=class_weight)


Epoch 1/160
160/160 [==============================] - 2s 4ms/step - loss: 2.4448 - accuracy: 0.5775
Epoch 2/160
160/160 [==============================] - 0s 2ms/step - loss: 1.7285 - accuracy: 0.5806
Epoch 3/160
160/160 [==============================] - 0s 2ms/step - loss: 1.1714 - accuracy: 0.5822
Epoch 4/160
160/160 [==============================] - 0s 2ms/step - loss: 1.1662 - accuracy: 0.5822
Epoch 5/160
160/160 [==============================] - 0s 2ms/step - loss: 0.5002 - accuracy: 0.5822
Epoch 6/160
160/160 [==============================] - 0s 2ms/step - loss: 0.4491 - accuracy: 0.5822
Epoch 7/160
160/160 [==============================] - 0s 2ms/step - loss: 0.4429 - accuracy: 0.5822
Epoch 8/160
160/160 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.5806
Epoch 9/160
160/160 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.5822
Epoch 10/160
160/160 [==============================] - 0s 2ms/step - loss: 0.4238 - accura

160/160 [==============================] - 0s 2ms/step - loss: 0.1187 - accuracy: 0.9233


In [18]:
ON_accuracy=[]
OFF_accuracy=[]

#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(1,35):
    
    #Conv1D Architecture
    model = Sequential()
    model.add(Conv1D(nb_filters, 1, padding="same", activation="relu",input_shape=(nbfeatures,binning)))
    model.add(Conv1D(15, 1, padding="same", activation="relu",input_shape=(nb_filters,binning)))
    model.add(Conv1D(9, 1, padding="same", activation="relu",input_shape=(15,binning)))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(binning, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(bigdata, Class_ON, test_size=0.33, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=70,batch_size=4,class_weight=class_weight)

    #Obtain the accuradcy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0

    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")

Epoch 1/70
153/153 [==============================] - 1s 2ms/step - loss: 1.9903 - accuracy: 0.9837
Epoch 2/70
153/153 [==============================] - 0s 2ms/step - loss: 1.8852 - accuracy: 0.9657
Epoch 3/70
153/153 [==============================] - 0s 2ms/step - loss: 1.1020 - accuracy: 0.5948
Epoch 4/70
153/153 [==============================] - 0s 2ms/step - loss: 1.0563 - accuracy: 0.5784
Epoch 5/70
153/153 [==============================] - 0s 2ms/step - loss: 0.6966 - accuracy: 0.5752
Epoch 6/70
153/153 [==============================] - 0s 2ms/step - loss: 0.5360 - accuracy: 0.5752
Epoch 7/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4599 - accuracy: 0.5768
Epoch 8/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4054 - accuracy: 0.5768
Epoch 9/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3811 - accuracy: 0.5784
Epoch 10/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3750 - accuracy: 0.5801

153/153 [==============================] - 0s 2ms/step - loss: 0.4436 - accuracy: 0.5784
Epoch 22/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.5784
Epoch 23/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.5784
Epoch 24/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4433 - accuracy: 0.5784
Epoch 25/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.5784
Epoch 26/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4435 - accuracy: 0.5784
Epoch 27/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4423 - accuracy: 0.5784
Epoch 28/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4441 - accuracy: 0.5784
Epoch 29/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4425 - accuracy: 0.5784
Epoch 30/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4427 - accuracy: 0.5784
Ep

153/153 [==============================] - 0s 2ms/step - loss: 0.1863 - accuracy: 0.9265
Epoch 42/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1758 - accuracy: 0.9314
Epoch 43/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1808 - accuracy: 0.9281
Epoch 44/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1572 - accuracy: 0.9363
Epoch 45/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1852 - accuracy: 0.9297
Epoch 46/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1936 - accuracy: 0.9265
Epoch 47/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1708 - accuracy: 0.9281
Epoch 48/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1740 - accuracy: 0.9314
Epoch 49/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1747 - accuracy: 0.9281
Epoch 50/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1733 - accuracy: 0.9281
Ep

153/153 [==============================] - 0s 2ms/step - loss: 0.3871 - accuracy: 0.5850
Epoch 62/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3835 - accuracy: 0.5850
Epoch 63/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3839 - accuracy: 0.5850
Epoch 64/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3834 - accuracy: 0.5850
Epoch 65/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3836 - accuracy: 0.5850
Epoch 66/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3831 - accuracy: 0.5850
Epoch 67/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3825 - accuracy: 0.5850
Epoch 68/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3837 - accuracy: 0.5850
Epoch 69/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3832 - accuracy: 0.5850
Epoch 70/70
10/10 [==============================] - 0s 2ms/step
Epoch 1/70
153/153 [==================

153/153 [==============================] - 0s 2ms/step - loss: 0.4466 - accuracy: 0.5784
Epoch 12/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4408 - accuracy: 0.5784
Epoch 13/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.5784
Epoch 14/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.5784
Epoch 15/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.5784
Epoch 16/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4154 - accuracy: 0.5784
Epoch 17/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4077 - accuracy: 0.5784
Epoch 18/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3982 - accuracy: 0.5784
Epoch 19/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3787 - accuracy: 0.5784
Epoch 20/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3471 - accuracy: 0.6797
Ep

...

153/153 [==============================] - 0s 3ms/step - loss: 0.1302 - accuracy: 0.9281
Epoch 62/70
153/153 [==============================] - 0s 3ms/step - loss: 0.1498 - accuracy: 0.9232
Epoch 63/70
153/153 [==============================] - 0s 3ms/step - loss: 0.1255 - accuracy: 0.9232
Epoch 64/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1354 - accuracy: 0.9248
Epoch 65/70
153/153 [==============================] - 0s 3ms/step - loss: 0.1202 - accuracy: 0.9265
Epoch 66/70
153/153 [==============================] - 0s 3ms/step - loss: 0.1332 - accuracy: 0.9281
Epoch 67/70
153/153 [==============================] - 0s 3ms/step - loss: 0.1141 - accuracy: 0.9248
Epoch 68/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1389 - accuracy: 0.9248
Epoch 69/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1209 - accuracy: 0.9216
Epoch 70/70
10/10 [==============================] - 0s 2ms/step
Epoch 1/70
153/153 [==================

153/153 [==============================] - 0s 3ms/step - loss: 0.4693 - accuracy: 0.5866
Epoch 12/70
153/153 [==============================] - 0s 3ms/step - loss: 0.4540 - accuracy: 0.5866
Epoch 13/70
153/153 [==============================] - 0s 3ms/step - loss: 0.4464 - accuracy: 0.5866
Epoch 14/70
153/153 [==============================] - 0s 3ms/step - loss: 0.4398 - accuracy: 0.5866
Epoch 15/70
153/153 [==============================] - 0s 3ms/step - loss: 0.4304 - accuracy: 0.5866
Epoch 16/70
153/153 [==============================] - 0s 3ms/step - loss: 0.4174 - accuracy: 0.5866
Epoch 17/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4034 - accuracy: 0.5882
Epoch 18/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3847 - accuracy: 0.5882
Epoch 19/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3677 - accuracy: 0.5882
Epoch 20/70
153/153 [==============================] - 0s 2ms/step - loss: 0.3543 - accuracy: 0.6699
Ep

153/153 [==============================] - 0s 2ms/step - loss: 0.2410 - accuracy: 0.8971
Epoch 32/70
153/153 [==============================] - 0s 2ms/step - loss: 0.2129 - accuracy: 0.9052
Epoch 33/70
153/153 [==============================] - 0s 2ms/step - loss: 0.2182 - accuracy: 0.9150
Epoch 34/70
153/153 [==============================] - 0s 2ms/step - loss: 0.2267 - accuracy: 0.9052
Epoch 35/70
153/153 [==============================] - 0s 2ms/step - loss: 0.2049 - accuracy: 0.9069
Epoch 36/70
153/153 [==============================] - 0s 2ms/step - loss: 0.2045 - accuracy: 0.9069
Epoch 37/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1973 - accuracy: 0.9085
Epoch 38/70
153/153 [==============================] - 0s 2ms/step - loss: 0.2043 - accuracy: 0.9101
Epoch 39/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1940 - accuracy: 0.9069
Epoch 40/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1909 - accuracy: 0.9085
Ep

153/153 [==============================] - 0s 2ms/step - loss: 0.1587 - accuracy: 0.9248
Epoch 52/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1614 - accuracy: 0.9248
Epoch 53/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.9232
Epoch 54/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1592 - accuracy: 0.9248
Epoch 55/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1678 - accuracy: 0.9248
Epoch 56/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1584 - accuracy: 0.9265
Epoch 57/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1544 - accuracy: 0.9232
Epoch 58/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1517 - accuracy: 0.9248
Epoch 59/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1444 - accuracy: 0.9265
Epoch 60/70
153/153 [==============================] - 0s 2ms/step - loss: 0.1526 - accuracy: 0.9248
Ep

153/153 [==============================] - 1s 2ms/step - loss: 0.6655 - accuracy: 0.7402
Epoch 2/70
153/153 [==============================] - 0s 2ms/step - loss: 0.9585 - accuracy: 0.5899
Epoch 3/70
153/153 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.5866
Epoch 4/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4979 - accuracy: 0.5882
Epoch 5/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4797 - accuracy: 0.5899
Epoch 6/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4662 - accuracy: 0.5899
Epoch 7/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4537 - accuracy: 0.5915
Epoch 8/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4388 - accuracy: 0.5899
Epoch 9/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.5899
Epoch 10/70
153/153 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.5899
Epoch 11/7